In [29]:
# !pip install --upgrade autogluon.multimodal


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.multimodal import MultiModalPredictor
from sklearn.utils import resample
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image

print("Bibliotecas importadas correctamente.")

Bibliotecas importadas correctamente.


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Directorio principal de imágenes en Google Drive
images_dir = 'arcgis-survey-images'

In [5]:
# Verificar si el directorio existe
if not os.path.isdir(images_dir):
    raise FileNotFoundError(f"El directorio '{images_dir}' no existe. Por favor, verifica la ruta.")

In [6]:
# Obtener las clases a partir de los nombres de los subdirectorios
class_names = sorted([d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))])
print(f"Clases encontradas: {class_names}")

# Recopilar rutas de imágenes y etiquetas
data = []
for class_label in class_names:
    class_dir = os.path.join(images_dir, class_label)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(class_dir, img_name)
            data.append({'image': img_path, 'label': class_label})

Clases encontradas: ['Chinche salivosa', 'Clororis', 'Hoja sana', 'Roya naranja', 'Roya purpura']


In [7]:
# Crear un DataFrame
df = pd.DataFrame(data)
print(f"Total de imágenes: {len(df)}")

Total de imágenes: 3289


In [8]:
# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=123)
print(f"Entrenamiento: {len(train_df)} imágenes")
print(f"Validación: {len(valid_df)} imágenes")

Entrenamiento: 2631 imágenes
Validación: 658 imágenes


In [9]:
# Manejo de clases desequilibradas en el conjunto de entrenamiento
class_counts = train_df['label'].value_counts()
print("Conteo de imágenes por clase en el conjunto de entrenamiento:")
print(class_counts)

Conteo de imágenes por clase en el conjunto de entrenamiento:
label
Roya naranja        801
Hoja sana           669
Chinche salivosa    660
Clororis            323
Roya purpura        178
Name: count, dtype: int64


In [10]:
# Encontrar la clase con el mayor número de muestras
max_count = class_counts.max()

In [11]:
# Aumentar las muestras de las clases minoritarias
dfs = []
for class_label, count in class_counts.items():
    df_class = train_df[train_df['label'] == class_label]
    if count < max_count:
        df_class = resample(df_class,
                            replace=True,     # Muestra con reemplazo
                            n_samples=max_count,  # Aumentar a max_count
                            random_state=123)
    dfs.append(df_class)

In [12]:
train_df_balanced = pd.concat(dfs)

In [13]:
# Mezclar el DataFrame balanceado
train_df_balanced = train_df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)
print("Datos de entrenamiento después del balanceo:")
print(train_df_balanced['label'].value_counts())

Datos de entrenamiento después del balanceo:
label
Roya purpura        801
Clororis            801
Roya naranja        801
Hoja sana           801
Chinche salivosa    801
Name: count, dtype: int64


In [14]:
# !python --version


In [15]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [16]:
# !pip uninstall torchvision 

In [17]:
# !pip uninstall torchaudio

In [18]:
# !pip show torch


In [19]:
import torch

# Verificar si hay una GPU disponible
if torch.cuda.is_available():
    num_gpus = 1  # Usar 1 GPU
    print("GPU disponible. Usando GPU para el entrenamiento.")
else:
    num_gpus = 0  # Usar CPU
    print("No se encontró GPU. Usando CPU para el entrenamiento.")

GPU disponible. Usando GPU para el entrenamiento.


In [20]:
# !nvcc --version

In [21]:
# !pip install --upgrade nvidia-ml-py3


In [22]:
# !nvidia-smi

In [23]:
# Definir el predictor de AutoMM para clasificación
predictor = MultiModalPredictor(problem_type="classification")

In [24]:
# import os
# import multiprocessing

# # Configurar Jupyter para usar un solo hilo
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
# os.environ['MKL_NUM_THREADS'] = '1'
# os.environ['OMP_NUM_THREADS'] = '1'

# # Configurar el método de inicio de multiprocesamiento
# multiprocessing.set_start_method('spawn', force=True)

In [27]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [26]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.3.1+cu118
True


In [28]:
!nvidia-smi


Wed Oct  2 16:24:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   46C    P8              5W /   50W |     597MiB /   4096MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
from autogluon.multimodal import MultiModalPredictor

# Inicializar el predictor
predictor = MultiModalPredictor(
    label='label',
    path='autogluon_plague_classifier',
    problem_type='multiclass',
    eval_metric='accuracy'
)

# Ajustar el modelo
predictor.fit(
    train_data=train_df_balanced,
    tuning_data=valid_df,
    time_limit=7200,
    presets="best_quality"
)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Pytorch Version:    2.3.1+cu118
CUDA Version:       11.8
Memory Avail:       19.63 GB / 31.37 GB (62.6%)
Disk Space Avail:   113.98 GB / 475.69 GB (24.0%)



AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir c:\Users\ealda\OneDrive\Documents\GitHub\Proyecto-Pantaleon\autogluon_plague_classifier
    ```

Seed set to 0


NVMLError_InvalidArgument: Invalid Argument

In [ ]:
# Evaluar el modelo en el conjunto de validación
print("Evaluando el modelo en el conjunto de validación...")
test_metrics = predictor.evaluate(valid_df)
print("Métricas de evaluación:", test_metrics)

In [ ]:
# Obtener predicciones en el conjunto de validación
print("Obteniendo predicciones en el conjunto de validación...")
valid_predictions = predictor.predict(valid_df)

In [ ]:
# Añadir las predicciones al DataFrame de validación
valid_df['pred_label'] = valid_predictions

In [ ]:
# Generar el reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(valid_df['label'], valid_df['pred_label'], target_names=class_names))

In [ ]:
# Crear la matriz de confusión
conf_matrix = confusion_matrix(valid_df['label'], valid_df['pred_label'], labels=class_names)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
# Mostrar algunas imágenes con sus predicciones
def show_predictions(predictor, df, num_images=5):
    samples = df.sample(n=num_images, random_state=42)
    for idx, row in samples.iterrows():
        img_path = row['image']
        true_label = row['label']
        image = Image.open(img_path).convert('RGB')

        pred_label = predictor.predict({'image': img_path})

        plt.figure(figsize=(4,4))
        plt.imshow(image)
        plt.title(f"Verdadero: {true_label}\nPredicción: {pred_label.values[0]}")
        plt.axis('off')
        plt.show()

In [ ]:
# Mostrar 5 predicciones aleatorias
show_predictions(predictor, valid_df, num_images=5)

In [ ]:
# Guardar el modelo
predictor.save("autogluon_plague_classifier")
print("Modelo guardado en 'autogluon_plague_classifier'.")

In [ ]:
# Cargar el modelo guardado con AutoMM
loaded_predictor = MultiModalPredictor.load("autogluon_plague_classifier")
print("Modelo cargado desde 'autogluon_plague_classifier'.")

In [ ]:
# Evaluar nuevamente para verificar
test_metrics = loaded_predictor.evaluate(valid_df)
print("Métricas de evaluación del modelo cargado:", test_metrics)